In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import joblib

In [ ]:
data=pd.concat([
    pd.read_csv('/content/Youtube01-Psy.csv'),
    pd.read_csv('/content/Youtube02-KatyPerry.csv'),
    pd.read_csv('/content/Youtube03-LMFAO.csv'),
    pd.read_csv('/content/Youtube04-Eminem.csv'),
    pd.read_csv('/content/Youtube05-Shakira.csv')
])

In [ ]:
data.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [ ]:
data.tail()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
365,_2viQ_Qnc6-bMSjqyL1NKj57ROicCSJV5SwTrw-RFFA,Katie Mettam,2013-07-13T13:27:39.441000,I love this song because we sing it at Camp al...,0
366,_2viQ_Qnc6-pY-1yR6K2FhmC5i48-WuNx5CumlHLDAI,Sabina Pearson-Smith,2013-07-13T13:14:30.021000,I love this song for two reasons: 1.it is abou...,0
367,_2viQ_Qnc6_k_n_Bse9zVhJP8tJReZpo8uM2uZfnzDs,jeffrey jules,2013-07-13T12:09:31.188000,wow,0
368,_2viQ_Qnc6_yBt8UGMWyg3vh0PulTqcqyQtdE7d4Fl0,Aishlin Maciel,2013-07-13T11:17:52.308000,Shakira u are so wiredo,0
369,_2viQ_Qnc685RPw1aSa1tfrIuHXRvAQ2rPT9R06KTqA,Latin Bosch,2013-07-12T22:33:27.916000,Shakira is the best dancer,0


In [ ]:
data.describe()

,CLASS
count,1956.000000
mean,0.513804
std,0.499937
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1956 entries, 0 to 369
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   COMMENT_ID  1956 non-null   object
 1   AUTHOR      1956 non-null   object
 2   DATE        1711 non-null   object
 3   CONTENT     1956 non-null   object
 4   CLASS       1956 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 91.7+ KB


In [ ]:
seed=1234
x_train,x_test,y_train,y_test=train_test_split(
    data['CONTENT'],
    data['CLASS'],
    test_size=0.2,
    random_state=seed
)

In [ ]:
x_train.shape

(1564,)

In [ ]:
y_train.shape

(1564,)

In [ ]:
x_test.shape

(392,)

In [ ]:
y_test.shape

(392,)

In [ ]:
pipe= make_pipeline(
    CountVectorizer(),
    RandomForestClassifier(random_state=seed)
)
print(pipe)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=1234))])


In [ ]:
pipe_fitted=pipe.fit(x_train,y_train)

In [ ]:
pipe_fitted

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=1234))])

In [ ]:
pipe_fitted.score(x_test,y_test)

0.9668367346938775

In [ ]:
pipe_fitted.predict(['what a great vidéo!'])

array([0])

In [ ]:
pipe_fitted.predict(['click here to win apple phone'])

array([0])

In [ ]:
scores=cross_val_score(pipe,x_train,y_train,cv=3)
scores

array([0.93103448, 0.96353167, 0.9462572 ])

In [ ]:
scores.mean()

0.9469411167736669

In [ ]:
scores.std()

0.0132757322822521

In [ ]:
#reclage des hyperparametres
parameters={
    'countvectorizer__max_features':(None,1000,2000),
    'countvectorizer__ngram_range':((1,1),(1,2)),
    'countvectorizer__stop_words': (None,'english'),
    'randomforestclassifier__n_estimators':(50,100,200)

}
print(parameters)

{'countvectorizer__max_features': (None, 1000, 2000), 'countvectorizer__ngram_range': ((1, 1), (1, 2)), 'countvectorizer__stop_words': (None, 'english'), 'randomforestclassifier__n_estimators': (50, 100, 200)}


In [ ]:
search_model=GridSearchCV(pipe,parameters,cv=3,n_jobs=-1)
search_model

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('countvectorizer', CountVectorizer()),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(random_state=1234))]),
             n_jobs=-1,
             param_grid={'countvectorizer__max_features': (None, 1000, 2000),
                         'countvectorizer__ngram_range': ((1, 1), (1, 2)),
                         'countvectorizer__stop_words': (None, 'english'),
                         'randomforestclassifier__n_estimators': (50, 100,
                                                                  200)})

In [ ]:
search_model.fit(x_train,y_train)
print(search_model.best_params_)
print(search_model.best_score_)

{'countvectorizer__max_features': 1000, 'countvectorizer__ngram_range': (1, 1), 'countvectorizer__stop_words': 'english', 'randomforestclassifier__n_estimators': 100}
0.9578102823188533


In [ ]:
joblib.dump(value=search_model.best_estimator_,filename='spam_detector_model.pkl')

['spam_detector_model.pkl']

In [ ]:
#chargement de modele
loaded_model=joblib.load(filename='spam_detector_model.pkl')

In [ ]:
loaded_model.predict(['thank you for this video'])

array([1])